# Using TensorBoard with AzureML.

In [ ]:
from azureml.core import Workspace, Dataset
from azureml.core import Experiment
import os
import shutil

# Connect to workspace.

In [ ]:
workspace = Workspace.from_config()
workspace

# Select experiment(s).

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

checkboxes = []

for experiment_name, experiment in workspace.experiments.items():
    checkbox = widgets.Checkbox(
        value=False,
        description=experiment_name
    )
    display(checkbox)
    checkboxes.append(checkbox)

# Download logs of selected experiments.

In [ ]:
# Get the selected experiments.
selected_experiments = []
for checkbox in checkboxes:
    if checkbox.value == True:
        selected_experiments.append(checkbox.description)

# Get folder.
temp_path = "logs"
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)
os.mkdir(temp_path)

# For each selected experiment download logs of all completed runs.
for selected_experiment in selected_experiments:
    print("Experiment: {}".format(selected_experiment))
    experiment = workspace.experiments.get(selected_experiment)
    for run_index, run in enumerate(list(experiment.get_runs())[::-1]):
        log_path = os.path.join(temp_path, experiment.name, "run_{:02d}".format(run_index + 1))
        if run.status == "Completed":
            print("  Run: {}".format(run_index + 1))
            run.download_files(prefix="logs", output_directory=log_path, output_paths=None, batch_size=100, append_prefix=False)

print("Done.")

# Start tensorboard.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs